In [1]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [2]:
time_slots = []
days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
hours = range(7, 23)
for day in days:
    for hour in hours:
        time_slot = f'{day}_{hour}'
        time_slots.append(time_slot)

In [3]:
activity_times = {
    'robotics':['Mon_15','Mon_16','Mon_17','Wed_15','Wed_16','Thu_15','Thu_16','Thu_17'],
    'track':['Mon_15','Mon_16','Tue_15','Tue_16','Wed_15','Wed_16','Thu_15','Thu_16','Fri_15','Fri_16','Sat_9','Sat_10'],
    'swim':['Mon_19','Mon_20','Tue_16','Tue_17','Tue_18','Wed_18','Wed_19','Wed_20','Fri_16','Fri_17','Fri_18','Sat_7','Sat_8','Sat_9','Sun_17','Sun_18','Sun_19'],
    'tutoring':['Thu_17','Thu_18'],
    'music':time_slots,
}

In [4]:
activity_vars = {}
for activity in activity_times:
    for time in activity_times[activity]:
        act_var = model.NewBoolVar(f'{activity}_{time}')
        activity_vars[(activity,time)] = act_var

In [5]:
for slot in time_slots:
    relevant_vars = [activity_vars[(act, slot)] for act in activity_times if (act, slot) in activity_vars]
    model.Add(sum(relevant_vars) <= 1)

In [6]:
target_hours = {'robotics': 7, 'track': 11, 'swim': 16, 'tutoring': 1, 'music': 2}
for act, target in target_hours.items():
    vars_for_activity = [activity_vars[(act,slot)] for slot in activity_times[act]]
    model.Add(sum(vars_for_activity)>= max(target-2,1))
    model.Add(sum(vars_for_activity)<=target+1)

In [7]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [8]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for slot in time_slots:
        for act in activity_times:
            if (act, slot) in activity_vars and solver.Value(activity_vars[(act,slot)]) == 1:
                print(f'{slot}: {act}')

Mon_15: robotics
Mon_16: robotics
Mon_17: robotics
Mon_19: swim
Mon_20: swim
Tue_15: track
Tue_16: track
Tue_17: swim
Tue_18: swim
Wed_15: robotics
Wed_16: track
Wed_18: swim
Wed_19: swim
Wed_20: swim
Thu_15: track
Thu_16: track
Thu_17: robotics
Thu_18: tutoring
Fri_15: track
Fri_16: track
Fri_17: swim
Fri_18: swim
Sat_7: swim
Sat_8: swim
Sat_9: track
Sat_10: track
Sun_17: swim
Sun_18: swim
Sun_19: swim
Sun_21: music
Sun_22: music


In [9]:
for act in target_hours:
    hours = sum(
        solver.Value(activity_vars[(act, slot)])
        for slot in activity_times[act]
        if (act, slot) in activity_vars
    )
    print(f"{act}: {hours} hours scheduled")

robotics: 5 hours scheduled
track: 9 hours scheduled
swim: 14 hours scheduled
tutoring: 1 hours scheduled
music: 2 hours scheduled
